In [1]:
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from vaderSentiment import vaderSentiment as vader 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
pd.options.display.max_columns = 999
from imblearn.over_sampling import SMOTE
import numpy as np
import sklearn
 
from sklearn import preprocessing
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expec

In [14]:
df = pd.read_csv('vaderized.csv')

In [16]:
labels = pd.read_csv('labels.csv')

In [17]:
df.head()

,Unnamed: 0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,review_10,review_11,review_12,review_13,review_14,review_15,review_16,review_17,review_18,review_19,review_20,neg1,neu1,pos1,compound1,neg2,neu2,pos2,compound2,neg3,neu3,pos3,compound3,neg4,neu4,pos4,compound4,neg5,neu5,pos5,compound5,neg6,neu6,pos6,compound6,neg7,neu7,pos7,compound7,neg8,neu8,pos8,compound8,neg9,neu9,pos9,compound9,neg10,neu10,pos10,compound10,neg11,neu11,pos11,compound11,neg12,neu12,pos12,compound12,neg13,neu13,pos13,compound13,neg14,neu14,pos14,compound14,neg15,neu15,pos15,compound15,neg16,neu16,pos16,compound16,neg17,neu17,pos17,compound17,neg18,neu18,pos18,compound18,neg19,neu19,pos19,compound19,neg20,neu20,pos20,compound20
0,0,Had a reservation on Saturday night for dinner...,Came here on a Friday night around 6:30pm and ...,Oh my goodness where do I even start? My boyfr...,"Unfortunately, dinner here was pretty disappoi...","This place is overrated, i would give only 2 s...","I've been here twice and both times, I was pre...",Great place. The waiter was extremely helpful ...,This place is an affordable gem! Perfect for a...,After hearing tons of people rave about this s...,Great seafood restaurant that is great value f...,Quick recap: come here if you like seafood. Or...,An unheard of trifecta: Cheap. Great tasting. ...,Sat at the bar and ordered the scallops with m...,"This is a cute spot to go to, especially for a...","Upstate specializes in seafood, and it's a ver...",I got here early with my SO on a Friday evenin...,"Okay, so Upstate does not need a review from m...",I've been meaning to try this place out for th...,I am so disappointed. Upstate has been on my l...,Waking into Upstate you think it's a small hol...,0.000,0.747,0.253,0.9652,0.000,0.692,0.308,0.9806,0.000,0.777,0.223,0.9862,0.085,0.769,0.146,0.9573,0.068,0.881,0.051,-0.3834,0.030,0.765,0.205,0.9814,0.033,0.585,0.381,0.9775,0.035,0.785,0.180,0.9928,0.051,0.732,0.217,0.9900,0.035,0.458,0.507,0.9833,0.021,0.722,0.257,0.9794,0.014,0.709,0.276,0.9989,0.000,0.826,0.174,0.9723,0.000,0.584,0.416,0.9899,0.030,0.735,0.234,0.9989,0.021,0.783,0.195,0.9817,0.023,0.820,0.157,0.8908,0.025,0.861,0.114,0.9166,0.035,0.754,0.211,0.9978,0.052,0.776,0.172,0.9914
1,1,The food and drinks here are both delicious. W...,My boyfriend took me here for my birthday earl...,We came for our anniversary dinner. I got the ...,Came here for a light dinner with my friend an...,Amelie is a lively wine bar with with cheese a...,This was my second time I visited the place. T...,I am simply in love. This has become my new fa...,- One of my favourite French restaurants in ny...,It really tasted like the last week of summer....,Meh. Pretty good. Middle of the road classic F...,I feel like it is very difficult to find a goo...,The happy hour deal can't be beat - wine fligh...,"This place is the BOMB!!! Eric, the manager, ...",Got my first wine flight here and I was not di...,I was in the area today and looked on yelp for...,Truly enjoyed our dining experience at Amelie....,We will be back again and again - we just had ...,"I just love this French place. I feel home, si...","This French wine bar is such a cozy spot, even...",My girlfriend took me here for my birthday din...,0.019,0.736,0.244,0.9762,0.038,0.734,0.228,0.9965,0.028,0.695,0.277,0.9956,0.000,0.557,0.443,0.9787,0.062,0.606,0.332,0.9765,0.078,0.743,0.179,0.9337,0.000,0.802,0.198,0.9950,0.000,0.836,0.164,0.9230,0.000,0.674,0.326,0.9789,0.031,0.738,0.231,0.8126,0.074,0.594,0.332,0.9956,0.020,0.702,0.278,0.9648,0.000,0.687,0.313,0.9753,0.021,0.627,0.352,0.9973,0.048,0.797,0.155,0.9277,0.040,0.770,0.190,0.9796,0.021,0.711,0.268,0.9938,0.000,0.599,0.401,0.9406,0.000,0.765,0.235,0.9861,0.049,0.802,0.149,0.9490
2,2,Was here with my boyfriend one Sunday evening ...,My friend and I checked out LoveMama around 6p...,We were vacationing in New York and found this...,This place is PACKED on a Friday night. Don't ...,Very cheap quality Thai

In [22]:
rva = ['review_'+ str(x+1) for x in range(20)]
comps = ['compound'+ str(x+1) for x in range(20)]

In [25]:
df.drop(columns=rva, inplace=True)

In [26]:

df.drop(columns=comps, inplace=True)

In [27]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [28]:
df.head()

,neg1,neu1,pos1,neg2,neu2,pos2,neg3,neu3,pos3,neg4,neu4,pos4,neg5,neu5,pos5,neg6,neu6,pos6,neg7,neu7,pos7,neg8,neu8,pos8,neg9,neu9,pos9,neg10,neu10,pos10,neg11,neu11,pos11,neg12,neu12,pos12,neg13,neu13,pos13,neg14,neu14,pos14,neg15,neu15,pos15,neg16,neu16,pos16,neg17,neu17,pos17,neg18,neu18,pos18,neg19,neu19,pos19,neg20,neu20,pos20
0,0.000,0.747,0.253,0.000,0.692,0.308,0.000,0.777,0.223,0.085,0.769,0.146,0.068,0.881,0.051,0.030,0.765,0.205,0.033,0.585,0.381,0.035,0.785,0.180,0.051,0.732,0.217,0.035,0.458,0.507,0.021,0.722,0.257,0.014,0.709,0.276,0.000,0.826,0.174,0.000,0.584,0.416,0.030,0.735,0.234,0.021,0.783,0.195,0.023,0.820,0.157,0.025,0.861,0.114,0.035,0.754,0.211,0.052,0.776,0.172
1,0.019,0.736,0.244,0.038,0.734,0.228,0.028,0.695,0.277,0.000,0.557,0.443,0.062,0.606,0.332,0.078,0.743,0.179,0.000,0.802,0.198,0.000,0.836,0.164,0.000,0.674,0.326,0.031,0.738,0.231,0.074,0.594,0.332,0.020,0.702,0.278,0.000,0.687,0.313,0.021,0.627,0.352,0.048,0.797,0.155,0.040,0.770,0.190,0.021,0.711,0.268,0.000,0.599,0.401,0.000,0.765,0.235,0.049,0.802,0.149
2,0.033,0.858,0.110,0.035,0.755,0.210,0.013,0.835,0.152,0.040,0.811,0.149,0.040,0.826,0.133,0.010,0.756,0.233,0.069,0.705,0.227,0.011,0.864,0.124,0.000,0.884,0.116,0.026,0.810,0.164,0.055,0.810,0.135,0.039,0.769,0.192,0.024,0.842,0.134,0.015,0.791,0.195,0.034,0.795,0.171,0.044,0.793,0.164,0.000,0.640,0.360,0.006,0.771,0.222,0.094,0.704,0.203,0.000,0.844,0.156
3,0.000,0.471,0.529,0.017,0.699,0.283,0.000,0.667,0.333,0.056,0.790,0.154,0.034,0.890,0.077,0.058,0.792,0.150,0.016,0.808,0.176,0.019,0.828,0.153,0.067,0.672,0.261,0.063,0.821,0.116,0.039,0.868,0.093,0.028,0.874,0.098,0.020,0.704,0.277,0.080,0.777,0.143,0.000,0.542,0.458,0.030,0.809,0.161,0.156,0.655,0.189,0.094,0.855,0.051,0.000,0.771,0.229,0.013,0.833,0.154
4,0.000,0.644,0.356,0.000,0.685,0.315,0.046,0.895,0.058,0.018,0.822,0.160,0.046,0.801,0.153,0.074,0.741,0.186,0.037,0.718,0.245,0.100,0.808,0.092,0.023,0.732,0.245,0.030,0.698,0.272,0.000,0.828,0.172,0.000,0.830,0.170,0.008,0.830,0.162,0.016,0.758,0.225,0.044,0.678,0.278,0.020,0.785,0.194,0.025,0.792,0.183,0.021,0.829,0.150,0.061,0.829,0.110,0.000,0.668,0.332


In [30]:
labels.rating = labels.rating.replace({3.0: 0, 5.0: 1, 3.5: 0, 4.0: 0, 4.5: 1})

In [32]:
labels.rating.value_counts()

0.0    1067
1.0     558
Name: rating, dtype: int64

In [37]:
labels.drop(columns=['Unnamed: 0'], inplace=True)

In [38]:
x = df
y = labels

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .2, random_state = 42)

In [40]:
print(y.rating.value_counts()) #Previous original class distribution
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_sample(x_train, y_train) 
print(pd.Series(y_train_resampled).value_counts()) #Preview synthetic sample class distribution

0.0    1067
1.0     558
Name: rating, dtype: int64
1.0    848
0.0    848
dtype: int64


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier 

In [45]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))

In [46]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy = 'uniform')
dummy.fit(x_train_resampled, y_train_resampled)
dummypred = dummy.predict(x_test)
print_metrics(y_test, dummypred)

Precision Score: 0.3107344632768362
Recall Score: 0.5188679245283019
Accuracy Score: 0.4676923076923077
F1 Score: 0.38869257950530034


In [49]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=8)
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
knnfit = knn.fit(x_train_resampled, y_train_resampled)
test_preds5 = knn.predict(x_test)
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))
    
print_metrics(y_test, test_preds5)

Precision Score: 0.35323383084577115
Recall Score: 0.6698113207547169
Accuracy Score: 0.49230769230769234
F1 Score: 0.4625407166123778


In [50]:
clf= DecisionTreeClassifier(criterion='entropy')
clf.fit(x_train_resampled,y_train_resampled) 
dtree_pred = clf.predict(x_test)

In [51]:
print_metrics(y_test,dtree_pred)

Precision Score: 0.35074626865671643
Recall Score: 0.44339622641509435
Accuracy Score: 0.5507692307692308
F1 Score: 0.3916666666666667


In [52]:
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), n_estimators=20)
clf = bagged_tree.fit(x_train_resampled, y_train_resampled)
bagged_pred = bagged_tree.predict(x_test)

In [53]:
print(classification_report(y_test, bagged_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.67      0.70       219
         1.0       0.41      0.48      0.44       106

    accuracy                           0.61       325
   macro avg       0.57      0.57      0.57       325
weighted avg       0.62      0.61      0.61       325



In [56]:
forest = RandomForestClassifier(n_estimators=100, max_depth= 8, max_leaf_nodes = 8)
forest.fit(x_train_resampled, y_train_resampled)

RFpred = forest.predict(x_test)
print(confusion_matrix(y_test, RFpred))
print(classification_report(y_test, RFpred))

[[161  58]
 [ 63  43]]
              precision    recall  f1-score   support

         0.0       0.72      0.74      0.73       219
         1.0       0.43      0.41      0.42       106

    accuracy                           0.63       325
   macro avg       0.57      0.57      0.57       325
weighted avg       0.62      0.63      0.63       325



In [57]:
adaboost_clf = AdaBoostClassifier()
gbt_clf = GradientBoostingClassifier(learning_rate =.02)

In [58]:
adaboost_clf.fit(x_train_resampled, y_train_resampled)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [59]:
gbt_clf.fit(x_train_resampled, y_train_resampled)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.02, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [60]:
adaboost_train_preds = adaboost_clf.predict(x_train_resampled)
adaboost_test_preds = adaboost_clf.predict(x_test)
gbt_clf_train_preds = gbt_clf.predict(x_train_resampled)
gbt_clf_test_preds = gbt_clf.predict(x_test)

In [61]:
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average = 'macro')
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
display_acc_and_f1_score(y_train_resampled, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train_resampled, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("Testing Metrics")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

Training Metrics
Model: AdaBoost
Accuracy: 0.7653301886792453
F1-Score: 0.7651995814688661

Model: Gradient Boosted Trees
Accuracy: 0.8030660377358491
F1-Score: 0.8027673516569988

Testing Metrics
Model: AdaBoost
Accuracy: 0.5784615384615385
F1-Score: 0.5467224546722455

Model: Gradient Boosted Trees
Accuracy: 0.5692307692307692
F1-Score: 0.5359794403198173


In [62]:
clf = xgb.XGBClassifier()
xgb = clf.fit(np.array(x_train_resampled), np.array(y_train_resampled))
training_preds = clf.predict(np.array(x_train_resampled))
xgb_preds = clf.predict(np.array(x_test))
training_accuracy = accuracy_score(np.array(y_train_resampled), training_preds)
val_accuracy = accuracy_score(np.array(y_test), xgb_preds)

In [63]:
print(classification_report(y_test, xgb_preds))

              precision    recall  f1-score   support

         0.0       0.72      0.74      0.73       219
         1.0       0.42      0.40      0.41       106

    accuracy                           0.62       325
   macro avg       0.57      0.57      0.57       325
weighted avg       0.62      0.62      0.62       325



In [76]:
from sklearn import svm

clf = svm.SVC(kernel='poly', C=5)
clf.fit(np.array(x_train_resampled), np.array(y_train_resampled))
svcy_pred = clf.predict(x_test)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [77]:
print(classification_report(y_test,svcy_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.63      0.68       219
         1.0       0.42      0.56      0.48       106

    accuracy                           0.61       325
   macro avg       0.58      0.59      0.58       325
weighted avg       0.64      0.61      0.62       325

